In [37]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [38]:
# My custom Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 16, 3,padding=1) #28
        self.batNorm1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1) #28
        self.batNorm2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1) #28
        self.batNorm3 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) # 14
        self.drop1 = nn.Dropout(0.20)
        
        self.conv4 = nn.Conv2d(16, 16, 3, padding=1) #14
        self.batNorm4 = nn.BatchNorm2d(16)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1) #14
        self.batNorm5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 16, 3, padding=1) #14
        self.batNorm6 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2) # 7
        self.drop2 = nn.Dropout(0.20)
        
        self.conv7 = nn.Conv2d(16, 16, 3, padding=1) #7
        self.batNorm7 = nn.BatchNorm2d(16)
        self.conv8 = nn.Conv2d(16, 16, 3, padding=1) #7
        self.batNorm8 = nn.BatchNorm2d(16)
        self.conv9 = nn.Conv2d(16, 16, 3, padding=1) #7
        self.batNorm9 = nn.BatchNorm2d(16)
        self.drop3 = nn.Dropout(0.15)
    
        self.fc1 = nn.Linear(16,10)


    def forward(self, x):
        x = self.batNorm1(F.relu(self.conv1(x)))
        x = self.batNorm2(F.relu(self.conv2(x)))
        x = self.batNorm3(F.relu(self.conv3(x)))
        x = self.drop1(self.pool1(x))
        
        x = self.batNorm4(F.relu(self.conv4(x)))
        x = self.batNorm5(F.relu(self.conv5(x)))
        x = self.batNorm6(F.relu(self.conv6(x)))
        x = self.drop2(self.pool2(x))
        
        x = self.batNorm7(F.relu(self.conv7(x)))
        x = self.batNorm8(F.relu(self.conv8(x)))
        x = self.batNorm9(F.relu(self.conv9(x)))
        x = self.drop3(x)
        
        # GAP layer
        x = F.adaptive_avg_pool2d(x, (1, 1)).view(x.size(0), -1)
        x = self.fc1(x)
        
        
        return F.log_softmax(x)

In [39]:
!pip install torchsummary
from torchsummary import summary

# run summary on cpu first then shift to device
model = Net().to("cpu")
summary(model, input_size=(1, 28, 28))

if torch.backends.mps.is_available():
    # Apple Silicon GPU
    device = "mps"
    use_cuda = True
elif torch.cuda.is_available():
    # nVidia GPU
    device = "cuda"
    use_cuda = True
else:
    # CPU
    device = "cpu"
print("Running models on: ", device)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 16, 28, 28]           2,320
       BatchNorm2d-4           [-1, 16, 28, 28]              32
            Conv2d-5           [-1, 16, 28, 28]           2,320
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
           Dropout-8           [-1, 16, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]           2,320
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,320
      BatchNorm2d-12           [-1, 16, 14, 14]              32
           Conv2d-13           [-1, 16, 14, 14]           2,320
      BatchNorm2d-14           [-1, 16,

/var/folders/6x/skj50cf15297r_q8sp6r6rwr0000gn/T/ipykernel_71893/3091036924.py:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [47]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [48]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [49]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

/var/folders/6x/skj50cf15297r_q8sp6r6rwr0000gn/T/ipykernel_71893/3091036924.py:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.13106082379817963 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.59it/s] 



Test set: Average loss: 0.0776, Accuracy: 9777/10000 (98%)



loss=0.08497384935617447 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 38.19it/s] 



Test set: Average loss: 0.0418, Accuracy: 9876/10000 (99%)



...


Test set: Average loss: 0.0197, Accuracy: 9938/10000 (99%)



loss=0.11501464247703552 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.93it/s]  



Test set: Average loss: 0.0181, Accuracy: 9937/10000 (99%)



loss=0.004924934823065996 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 35.60it/s] 



Test set: Average loss: 0.0194, Accuracy: 9945/10000 (99%)

